In [ ]:
import traceback

from data.data import DataProcessor
from evaluation.evaluation import ResultsManager
from experiments.resnet_experiment import TabularResNetExperiment

/home/gabriel/Research/tabular-resnet/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
CACHE_DIR = "./data/openml_cache"
RESULTS_DIR = "./results/resnet"
MODEL_NAME = "resnet"

SEED = 123
N_TRIALS = 50  # Optuna trials
CV_FOLDS = 10  # Cross-validation folds


def main():
    print("=" * 80)
    print("TABULAR RESNET - EXPERIMENT PIPELINE")
    print("=" * 80)
    print(f"Model: {MODEL_NAME}")
    print(f"Seed: {SEED}")
    print(f"Optuna trials: {N_TRIALS}")
    print(f"CV folds: {CV_FOLDS}")
    print(f"Results directory: {RESULTS_DIR}")
    print("=" * 80)

    # Initialize data processor and results manager
    processor = DataProcessor(seed=SEED, cache_dir=CACHE_DIR)
    results_manager = ResultsManager(save_dir=RESULTS_DIR, model_name=MODEL_NAME)

    # Find all processed PKL files
    pkl_files = list(processor.cache_dir.glob("*_dataset.pkl"))

    if not pkl_files:
        print("\nNo processed datasets found!")
        print(f"Looking in: {processor.cache_dir}")
        print("\nPlease run one of these first:")
        print("  1. python data.py (Pattern 1: to download and process)")
        print("  2. python data.py (Pattern 2: to process from raw CSVs)")
        return

    print(f"\n✓ Found {len(pkl_files)} processed datasets\n")

    # Run experiments on all datasets
    failed_datasets = []

    for idx, pkl_file in enumerate(pkl_files, 1):
        # Extract dataset name
        dataset_name = pkl_file.stem.replace("_dataset", "")

        print(f"\n{'#' * 80}")
        print(f"DATASET {idx}/{len(pkl_files)}: {dataset_name}")
        print(f"{'#' * 80}")

        try:
            # Load processed dataset (numpy arrays)
            X_train, y_train, X_test, y_test = processor.load_or_process_dataset(
                name=dataset_name
            )

            # Get number of classes
            n_classes = len(set(y_train))

            # Create experiment
            experiment = TabularResNetExperiment(
                dataset_name=dataset_name,
                X_train=X_train,
                y_train=y_train,
                X_test=X_test,
                y_test=y_test,
                n_classes=n_classes,
                n_trials=N_TRIALS,
                cv_folds=CV_FOLDS,
                seed=SEED,
            )

            # Run complete experiment
            results = experiment.run_complete_experiment()

            # Extract metrics and info for saving
            metrics = {
                "accuracy": results["accuracy"],
                "auc_ovo": results["auc_ovo"],
                "gmean": results["gmean"],
                "cross_entropy": results["cross_entropy"],
            }

            dataset_info = {
                "n_samples_train": results["n_samples_train"],
                "n_samples_test": results["n_samples_test"],
                "n_features": results["n_features"],
                "n_classes": results["n_classes"],
            }

            timings = {
                "tuning_time": results["tuning_time"],
                "training_time": results["training_time"],
                "prediction_time": results["prediction_time"],
                "total_time": results["total_time"],
            }

            # Save results
            results_manager.save_dataset_result(
                dataset_name=dataset_name,
                metrics=metrics,
                dataset_info=dataset_info,
                timings=timings,
                hyperparameters=results["best_params"],
            )

            print(f"\n✓ Completed {dataset_name} ({idx}/{len(pkl_files)})")

        except Exception as e:
            print(f"\n✗ Failed: {dataset_name}")
            print(f"   Error: {e}")
            failed_datasets.append((dataset_name, e))
            traceback.print_exc()
            continue

    # ========================================================================
    # Save final results
    # ========================================================================
    print("\n" + "=" * 80)
    print("SAVING FINAL RESULTS")
    print("=" * 80)

    # Save metrics CSV (for hypothesis testing)
    results_manager.save_metrics_csv()

    # Print final summary
    results_manager.print_summary()

    # Print failed datasets if any
    if failed_datasets:
        print(f"\nFailed datasets ({len(failed_datasets)}):")
        for dataset, error in failed_datasets:
            print(f"  - {dataset}: {error}")

    print("\n" + "=" * 80)
    print("EXPERIMENT COMPLETE!")
    print("=" * 80)
    print(f"✓ Total datasets: {len(pkl_files)}")
    print(f"✓ Successful: {len(results_manager.all_results)}")
    print(f"✓ Failed: {len(failed_datasets)}")
    print(f"\n✓ Metrics CSV saved: {RESULTS_DIR}/{MODEL_NAME}_metrics.csv")
    print("=" * 80)

In [3]:
main()

TABULAR RESNET - EXPERIMENT PIPELINE
Model: resnet
Seed: 123
Optuna trials: 50
CV folds: 10
Results directory: ./results/resnet

✓ Found 30 processed datasets


################################################################################
DATASET 1/30: MiceProtein
################################################################################
Loading processed dataset from cache: MiceProtein
Using device: cuda
Dataset: MiceProtein
  Train: (756, 77), Test: (324, 77)
  Features: 77, Classes: 8


[I 2025-11-26 23:32:05,557] A new study created in memory with name: no-name-7e345a0e-abda-401a-887f-2e2cd25ff4ea


✓ Data loaded to cuda

EXPERIMENT: MiceProtein

Starting hyperparameter optimization...
  Trials: 50, CV Folds: 10


[I 2025-11-26 23:32:15,050] Trial 0 finished with value: 0.9750431563456203 and parameters: {'d': 64, 'd_hidden_factor': 3.8778758791422523, 'n_layers': 5, 'hidden_dropout': 0.49038209919230774, 'residual_dropout': 0.34241486929243165, 'learning_rate': 0.00027720158198153483, 'batch_size': 128, 'epochs': 40}. Best is trial 0 with value: 0.9750431563456203.
[I 2025-11-26 23:32:42,050] Trial 1 finished with value: 0.9905003006155368 and parameters: {'d': 128, 'd_hidden_factor': 3.1262054953673535, 'n_layers': 6, 'hidden_dropout': 0.31720047927566053, 'residual_dropout': 0.4247158970388948, 'learning_rate': 0.0014906166728108333, 'batch_size': 64, 'epochs': 69}. Best is trial 1 with value: 0.9905003006155368.
[I 2025-11-26 23:33:02,677] Trial 2 finished with value: 0.9975073196879075 and parameters: {'d': 128, 'd_hidden_factor': 2.723451053318575, 'n_layers': 5, 'hidden_dropout': 0.212915145147914, 'residual_dropout': 0.15613061148623264, 'learning_rate': 0.0001901314327577386, 'batch_siz


✓ Optimization complete! Time: 5354.08s
  Best CV G-Mean: 1.0000
  Best parameters:
    d: 512
    d_hidden_factor: 4.323646972058734
    n_layers: 10
    hidden_dropout: 0.1778976046973269
    residual_dropout: 0.18698998490625918
    learning_rate: 1.1452211301313298e-05
    batch_size: 128
    epochs: 136

Training final model...
    Epoch 10/136: Train Loss = 0.7731, Val Loss = 0.7337
    Epoch 20/136: Train Loss = 0.3239, Val Loss = 0.3143
    Epoch 30/136: Train Loss = 0.1632, Val Loss = 0.1508
    Epoch 40/136: Train Loss = 0.0960, Val Loss = 0.0822
    Epoch 50/136: Train Loss = 0.0665, Val Loss = 0.0534
    Epoch 60/136: Train Loss = 0.0503, Val Loss = 0.0402
    Epoch 70/136: Train Loss = 0.0368, Val Loss = 0.0317
    Epoch 80/136: Train Loss = 0.0343, Val Loss = 0.0249
    Epoch 90/136: Train Loss = 0.0285, Val Loss = 0.0219
    Epoch 100/136: Train Loss = 0.0223, Val Loss = 0.0182
    Epoch 110/136: Train Loss = 0.0204, Val Loss = 0.0163
    Epoch 120/136: Train Loss = 0.0

...

    Early stopping at epoch 16
✓ Training complete! Time: 0.74s

Evaluating on test set...

RESULTS FOR dresses-sales
Accuracy:        0.5000
AUC OVO:         0.5864
G-Mean:          0.4480
Cross-Entropy:   0.7484
✓ Saved results for dresses-sales

✓ Completed dresses-sales (28/30)

################################################################################
DATASET 29/30: balance-scale
################################################################################
Loading processed dataset from cache: balance-scale
Using device: cuda
Dataset: balance-scale
  Train: (437, 4), Test: (188, 4)
  Features: 4, Classes: 3
✓ Data loaded to cuda

EXPERIMENT: balance-scale

Starting hyperparameter optimization...
  Trials: 50, CV Folds: 10


[I 2025-11-27 13:19:40,070] Trial 0 finished with value: 0.06933612743506347 and parameters: {'d': 64, 'd_hidden_factor': 3.8778758791422523, 'n_layers': 5, 'hidden_dropout': 0.49038209919230774, 'residual_dropout': 0.34241486929243165, 'learning_rate': 0.00027720158198153483, 'batch_size': 128, 'epochs': 40}. Best is trial 0 with value: 0.06933612743506347.
[I 2025-11-27 13:19:51,202] Trial 1 finished with value: 0.7839030731129073 and parameters: {'d': 128, 'd_hidden_factor': 3.1262054953673535, 'n_layers': 6, 'hidden_dropout': 0.31720047927566053, 'residual_dropout': 0.4247158970388948, 'learning_rate': 0.0014906166728108333, 'batch_size': 64, 'epochs': 69}. Best is trial 1 with value: 0.7839030731129073.
[I 2025-11-27 13:20:03,162] Trial 2 finished with value: 0.8038333749976619 and parameters: {'d': 128, 'd_hidden_factor': 2.723451053318575, 'n_layers': 5, 'hidden_dropout': 0.212915145147914, 'residual_dropout': 0.15613061148623264, 'learning_rate': 0.0001901314327577386, 'batch_s


✓ Optimization complete! Time: 503.65s
  Best CV G-Mean: 0.9510
  Best parameters:
    d: 256
    d_hidden_factor: 4.927212135393402
    n_layers: 1
    hidden_dropout: 0.4065623125080101
    residual_dropout: 0.005739073876571299
    learning_rate: 0.00852277869190327
    batch_size: 256
    epochs: 170

Training final model...
    Epoch 10/170: Train Loss = 0.1501, Val Loss = 0.1845
    Epoch 20/170: Train Loss = 0.1287, Val Loss = 0.1317
    Epoch 30/170: Train Loss = 0.1083, Val Loss = 0.1438
    Epoch 40/170: Train Loss = 0.0693, Val Loss = 0.0704
    Epoch 50/170: Train Loss = 0.0497, Val Loss = 0.0647
    Epoch 60/170: Train Loss = 0.0350, Val Loss = 0.0288
    Epoch 70/170: Train Loss = 0.0541, Val Loss = 0.0397
    Early stopping at epoch 73
✓ Training complete! Time: 0.17s

Evaluating on test set...

RESULTS FOR balance-scale
Accuracy:        0.9787
AUC OVO:         0.9968
G-Mean:          0.9659
Cross-Entropy:   0.0707
✓ Saved results for balance-scale

✓ Completed balance-

[I 2025-11-27 13:28:05,605] Trial 0 finished with value: 0.9975753094846148 and parameters: {'d': 64, 'd_hidden_factor': 3.8778758791422523, 'n_layers': 5, 'hidden_dropout': 0.49038209919230774, 'residual_dropout': 0.34241486929243165, 'learning_rate': 0.00027720158198153483, 'batch_size': 128, 'epochs': 40}. Best is trial 0 with value: 0.9975753094846148.
[I 2025-11-27 13:28:25,676] Trial 1 finished with value: 0.9954235137479704 and parameters: {'d': 128, 'd_hidden_factor': 3.1262054953673535, 'n_layers': 6, 'hidden_dropout': 0.31720047927566053, 'residual_dropout': 0.4247158970388948, 'learning_rate': 0.0014906166728108333, 'batch_size': 64, 'epochs': 69}. Best is trial 0 with value: 0.9975753094846148.
[I 2025-11-27 13:28:42,319] Trial 2 finished with value: 0.9966358590056631 and parameters: {'d': 128, 'd_hidden_factor': 2.723451053318575, 'n_layers': 5, 'hidden_dropout': 0.212915145147914, 'residual_dropout': 0.15613061148623264, 'learning_rate': 0.0001901314327577386, 'batch_siz


✓ Optimization complete! Time: 4799.10s
  Best CV G-Mean: 1.0000
  Best parameters:
    d: 64
    d_hidden_factor: 2.1444073493643163
    n_layers: 8
    hidden_dropout: 0.39773290288811947
    residual_dropout: 0.02489923440104569
    learning_rate: 6.731743934331164e-05
    batch_size: 32
    epochs: 146

Training final model...
    Epoch 10/146: Train Loss = 0.6102, Val Loss = 0.5120
    Epoch 20/146: Train Loss = 0.3209, Val Loss = 0.2760
    Epoch 30/146: Train Loss = 0.2039, Val Loss = 0.1956
    Epoch 40/146: Train Loss = 0.1517, Val Loss = 0.1337
    Epoch 50/146: Train Loss = 0.1163, Val Loss = 0.1005
    Epoch 60/146: Train Loss = 0.1006, Val Loss = 0.0773
    Epoch 70/146: Train Loss = 0.0760, Val Loss = 0.0619
    Epoch 80/146: Train Loss = 0.0682, Val Loss = 0.0495
    Epoch 90/146: Train Loss = 0.0671, Val Loss = 0.0433
    Epoch 100/146: Train Loss = 0.0353, Val Loss = 0.0347
    Epoch 110/146: Train Loss = 0.0521, Val Loss = 0.0296
    Epoch 120/146: Train Loss = 0.031